#  IA para Redes de Suministro 

👤 **Autor:** John Leonardo Vargas Mesa  
🔗 [LinkedIn](https://www.linkedin.com/in/leonardovargas/) | [GitHub](https://github.com/LeStark)  

## 📂 Repositorio en GitHub  
- 📓 **Notebooks:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/02%20-%20IA4SC)  
- 📑 **Data sets:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/00%20-%20Data/02%20-%20SC)  
---

# 📘 Notebook 5A  Uso de los modelos entrenados

# 🤖 Notebook 5A – Uso de los Modelos Entrenados

Este notebook tiene como propósito mostrar cómo **utilizar un modelo de análisis de sentimientos previamente entrenado** para realizar predicciones sobre nuevos conjuntos de reseñas.  
A partir del modelo desarrollado en el *Notebook 05 – Introducción al NLP*, se construye un flujo reproducible para limpiar, vectorizar y clasificar texto sin necesidad de reentrenar el modelo.

## 🧩 **Contenido del Notebook**

1. **Carga de los Modelos Entrenados:**  
   Se importan los objetos generados durante el entrenamiento:  
   - `sentiment_model.pkl` → modelo de clasificación entrenado.  
   - `tfidf_vectorizer.pkl` → vectorizador TF-IDF.  
   - `label_encoder.pkl` → codificador de etiquetas.  

   Estos permiten replicar el proceso completo sin volver a ejecutar el entrenamiento.

2. **Definición del Preprocesamiento:**  
   Se implementa la misma función `preprocess_text()` utilizada en el entrenamiento, que:
   - Convierte texto a minúsculas.  
   - Elimina caracteres no alfabéticos, números y URLs.  
   - Tokeniza, elimina *stopwords* y aplica lematización.  

   Esto asegura la **consistencia** entre el entrenamiento y la inferencia.

3. **Creación de la Función de Predicción (`predict_sentiment`):**  
   Se desarrolla una función integral que:
   - Limpia y normaliza el texto.  
   - Lo transforma con el vectorizador TF-IDF.  
   - Predice el sentimiento con el modelo cargado.  
   - Decodifica la salida numérica a etiquetas de texto (*positive*, *neutral*, *negative*).

4. **Prueba del Modelo con Nuevas Reseñas:**  
   Se aplican ejemplos de texto reales para observar las predicciones y verificar que el modelo generaliza correctamente.

5. **Aplicación Masiva sobre un DataFrame:**  
   Se carga un archivo CSV con reseñas nuevas (`sample_amazon_reviews.csv`)  
   y se aplica la función `predict_sentiment()` sobre toda la columna `reviewText`, generando una nueva columna `predicted_sentiment`.


## 🎯 **Objetivo de Aprendizaje**

El estudiante comprenderá cómo **reutilizar modelos de NLP ya entrenados** y cómo integrarlos en flujos de predicción para analizar grandes volúmenes de texto.  
Además, aprenderá a mantener la coherencia del pipeline (preprocesamiento → vectorización → predicción → decodificación) al aplicar el modelo en nuevos contextos.

## 🧾 **Herramientas Utilizadas**
- **joblib** → para cargar los modelos y objetos guardados.  
- **NLTK / spaCy** → preprocesamiento del texto.  
- **scikit-learn** → vectorización y predicción.  
- **pandas** → manipulación de datos.  

💡 *Este notebook demuestra cómo llevar un modelo de NLP desde su entrenamiento hasta su aplicación práctica, preparando el terreno para integraciones en sistemas reales o aplicaciones de análisis automatizado de texto.*


In [12]:
import pandas as pd
import numpy as np  
import joblib

# --- Procesamiento de texto ---
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# --- Descarga de recursos NLTK ---
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# --- Carga del modelo de lenguaje de spaCy ---
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jlvar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jlvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jlvar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jlvar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### **Carga del Archivo de Reseñas de Muestra**

Se carga un archivo CSV con una muestra de 100 reseñas de Amazon desde una URL pública de GitHub para realizar pruebas rápidas con el modelo entrenado.


In [4]:
url= r"https://raw.githubusercontent.com/LeStark/Cursos/refs/heads/main/00%20-%20Data/03%20-%20NLP/sample_amazon_reviews.csv"
data = pd.read_csv(url)
data.head()

,reviewText
0,When I opened the micro disc and adapter I did...
1,I purchased this product knowing there might b...
2,Bought October 7th. Passed away December 23rd....
3,This was an inexpensive way to get my Galaxy N...
4,"Works well, and fast, does everything it claim..."


###  **Carga del Modelo y Creación de la Función de Predicción**

En esta sección se cargan los objetos entrenados previamente para realizar predicciones sobre nuevas reseñas de texto.  
Estos objetos fueron generados durante la etapa de entrenamiento y se almacenaron en la carpeta `Models/`:

- **`sentiment_model.pkl`** → Modelo de clasificación entrenado (Regresión Logística).  
- **`tfidf_vectorizer.pkl`** → Vectorizador TF-IDF que transforma el texto en representaciones numéricas.  
- **`label_encoder.pkl`** → Codificador de etiquetas que traduce las categorías (*positive*, *neutral*, *negative*) a valores numéricos y viceversa.  

Además, se define nuevamente la función `preprocess_text()` para limpiar y normalizar el texto con el mismo proceso utilizado en el entrenamiento (minúsculas, eliminación de ruido, tokenización, stopwords y lematización).

Finalmente, la función `predict_sentiment()` integra todo el flujo:
1. Preprocesa el texto nuevo.  
2. Lo transforma con el vectorizador TF-IDF original.  
3. Predice el sentimiento utilizando el modelo entrenado.  
4. Decodifica el resultado para obtener la etiqueta final (**positive**, **neutral** o **negative**).

Esta función permitirá aplicar el modelo de análisis de sentimientos sobre cualquier conjunto nuevo de reseñas.


In [14]:

# --- Cargar los objetos entrenados ---

model = joblib.load("Models/sentiment_model.pkl")
vectorizer = joblib.load("Models/tfidf_vectorizer.pkl")
le = joblib.load("Models/label_encoder.pkl")
stop_words = set(stopwords.words("english"))

# --- Función de preprocesamiento (idéntica a la usada en el entrenamiento) ---
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    doc = nlp(" ".join(tokens))
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

# --- Función principal para predecir sentimiento ---
def predict_sentiment(text):
    """
    Recibe una reseña (o texto en bruto),
    aplica el mismo preprocesamiento usado en el entrenamiento,
    transforma el texto en vector TF-IDF y devuelve el sentimiento predicho.
    """
    # 1️ Limpiar el texto
    clean_text = preprocess_text(text)

    # 2️ Vectorizar usando el TF-IDF original
    vector = vectorizer.transform([clean_text])

    # 3️ Predecir usando el modelo cargado
    pred = model.predict(vector)

    # 4️ Decodificar la etiqueta numérica
    sentiment = le.inverse_transform(pred)[0]
    
    return sentiment



### **Aplicación del Modelo a Nuevas Reseñas**

Se aplica la función `predict_sentiment()` a la columna `reviewText` del DataFrame para predecir el sentimiento de cada reseña y visualizar los primeros resultados.


In [19]:
# Aplicar la función sobre una columna de un DataFrame
data["predicted_sentiment"] = data["reviewText"].apply(predict_sentiment)

# Ver resultados
data[["reviewText", "predicted_sentiment"]].head()

,reviewText,predicted_sentiment
0,When I opened the micro disc and adapter I did...,positive
1,I purchased this product knowing there might b...,negative
2,Bought October 7th. Passed away December 23rd....,negative
3,This was an inexpensive way to get my Galaxy N...,positive
4,"Works well, and fast, does everything it claim...",positive


In [ ]:

print(data.iloc[1]["reviewText"])
print(data.iloc[1]["predicted_sentiment"])